<h1> Use this notebook for pre-processing commonvoices dataset for ASR </h1>

<h2> Installation on SageMaker Instances </h2>
<h3>Old process : </h3>
    <ul>
        <li>Uinsg terminal, create a new conda env. python 3.7.12</li>
        <li>Run bash command to enable bash</li>
        <li>Activate the created env.   and install ipykernel</li>
        <li>Wait (a minute or two) so the new env. appears in UI Luncher</li> 
    </ul>
    <h3> New AMIs </h3>
   <ul><li>     Select conda_pytorch_38  and you are done with this step </li></ul>
   

<h2> If using Old process run the following cell</h2>


In [40]:
process_selected = False

while (not process_selected):
    select_Process = input("which method are you setting up with: old method or new AMIs?\
                \nChoose from 'OLD' or 'New'" )
    if (select_Process.lower() =='old' or select_Process.lower()=='new'):
        process_selected = True
        print('selecting process\n')
        print('process {} is selected'.format(select_Process))
    else:
        print('Either process not selected or it is not old or new')
            
    

which method are you setting up with: old method or new AMIs?                
Choose from 'OLD' or 'New' new


selecting process

process new is selected


In [44]:
%%writefile old_requirements.sh

conda install -y -c conda-forge ipywidgets
jupyter nbextension enable --py widgetsnbextension
conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
conda install -y pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
pip install      datasets==1.13.3
pip install      transformers==4.11.3
pip install      librosa
pip install      jiwer
pip install      s3fs

Writing old_requirements.sh


In [48]:
%%writefile new_requirements.sh

pip install datasets==1.13.3
echo 'datasets installed'

pip install transformers==4.11.3
echo 'transformers installed'

pip install librosa
echo 'librosa installed'

pip install jiwer
echo 'jiwer installed'

pip install s3fs
echo 's3fs installed'

pip install boto3
echo 'boto3 installed'

Overwriting new_requirements.sh


In [61]:
%%bash -s "$select_Process"

echo ${1}

if [[ ${1} == 'old' ]]
then
    echo "Installing old requirements"
    ./old_requirements.sh
elif [[ ${1} == 'new' ]]
then
    echo "Installing new requirements"
    ./new_requirements.sh
    
fi

new
Installing new requirements
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
datasets installed
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
transformers installed
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
librosa installed
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
jiwer installed
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
s3fs installed
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
boto3 installed


<h1> Configure AWS account </h1>
use <span style="color:red">configure aws</span> in terminal

<h1> Create folder names </h1>

### selec language

<span style="color:red">TO DO</span>: create a json file for language names <br>

<h4 style="color:red"> langauges to choose from </h4>

Languages: 
ab /ar /as /br /ca /cnh /cs
/cv/cy/de/dv/el/en/eo/es/et/eu/fa/fi/fr/fy-NL/ga-IE/hi/hsb/hu/ia/id/it/ja/ka/kab/ky/lg/lt/lv/mn/mt/nl/or/pa-IN/pl/pt/rm-sursilv/rm-vallader/ro/ru/rw/sah/sl/sv-SE/ta/th/tr/tt/uk/vi/vot/zh-CN/zh-HK/zh-TW

In [97]:
lang  = input('what language are you working with?/n select abbreviation from above' )
if (lang=='sl'):
    language = 'sloveni'
else:
    print('languate un-known')
    
    

what language are you working with?/n select abbreviation from above sl


#### Set up root path 

In [98]:
root = '/home/ec2-user/SageMaker/STT'

if (not os.path.isdir(os.path.join(root,language))):
    os.mkdir(os.path.join(root,language))
    

    

In [99]:
bucket_name       = 'commonvoicesdataset'
s3_train_fld      = 'train'
s3_validation_fld = 'validation'
check_points      = 'checkpoints'
last_model        = 'model'
dataset_dl_fld    = 'dataset'
dl_language       = lang

In [100]:
#cache dir is where we downlaod the dataset to
cache_dir= os.path.join(root,language,dataset_dl_fld)
if (os.path.isdir(cache_dir)):
    pass
else:
    os.mkdir(cache_dir)
    

In [101]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice",  lang,  split="train+validation",cache_dir =cache_dir)
common_voice_test  = load_dataset("common_voice" , lang, split="test",           cache_dir   =cache_dir)

Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /home/ec2-user/SageMaker/STT/sloveni/dataset/common_voice/sl/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9. Subsequent calls will reuse this data.


Reusing dataset common_voice (/home/ec2-user/SageMaker/STT/sloveni/dataset/common_voice/sl/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


documents have (i) which is bacially small l but when its convert or fount by textracts irts treat like cap
